# Project Objective

## Movie Recommendation

In this project I'm going to make a webpage where one can search with a movie name (available in the list) and the programe will recommend 5 similar movie names with poster picture automatically.

# Data Source

- https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?resource=download&select=movies_metadata.csv

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
credits_df = pd.read_csv("archive/credits.csv")

In [3]:
keywords_df = pd.read_csv("archive/keywords.csv")

In [4]:
movies_df = pd.read_csv("archive/movies_metadata.csv")

/tmp/ipykernel_13577/369803085.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv("archive/movies_metadata.csv")


In [5]:
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [6]:
credits_df.dtypes

cast    object
crew    object
id       int64
dtype: object

## Observations:

- All 3 features are usefull from our project perspective.
- We just need first 3 out of all cast members.
- From crew column we need the Director's name only.

In [7]:
keywords_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [8]:
keywords_df.dtypes

id           int64
keywords    object
dtype: object

## Observations:

- from keywords column we need to extract usefull keywords only.

In [9]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [10]:
movies_df.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [11]:
for i in movies_df.columns:
    print(i, movies_df[i][28], '='*100, sep='\n')

adult
False
belongs_to_collection
nan
budget
18000000
genres
[{'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 12, 'name': 'Adventure'}]
homepage
nan
id
902
imdb_id
tt0112682
original_language
fr
original_title
La Cité des Enfants Perdus
overview
A scientist in a surrealist society kidnaps children to steal their dreams, hoping that they slow his aging process.
popularity
9.822423
poster_path
/eVo6ewq4akfyJYy3GXkMsLNzEJc.jpg
production_companies
[{'name': 'Procirep', 'id': 311}, {'name': 'Constellation Productions', 'id': 590}, {'name': 'France 3 Cinéma', 'id': 591}, {'name': 'Claudie Ossard Productions', 'id': 592}, {'name': 'Eurimages', 'id': 850}, {'name': 'MEDIA Programme of the European Union', 'id': 851}, {'name': 'Cofimage 5', 'id': 1871}, {'name': 'Televisión Española (TVE)', 'id': 6639}, {'name': 'Tele München Fernseh Produktionsgesellschaft (TMG)', 'id': 7237}, {'name': "Club d'Investissement Média", 'id': 8170}, {'name': 'Canal+ España', 'id': 93

In [12]:
movies_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

## Observations:

- From 'generes' column we need the 'name' out of each dict.
- From 'production_companies' we will keep first 3 only
- We will keep the following columns as they are going to play the key role in predictionfor this particular project 
['adult','genres', 'id', 'overview', 'production_companies', 'title']

# Pre-processing

In [13]:
# keeping only the necessary columns in movies_df
movies_df = movies_df[['adult','genres', 'id', 'overview', 'production_companies', 'title']]

### Merge 3 dataframes on the 'id' column

In [14]:
movies_df.shape

(45466, 6)

In [15]:
credits_df.shape

(45476, 3)

In [16]:
keywords_df.shape

(46419, 2)

In [17]:
movies_df.columns

Index(['adult', 'genres', 'id', 'overview', 'production_companies', 'title'], dtype='object')

In [18]:
credits_df.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [19]:
keywords_df.columns

Index(['id', 'keywords'], dtype='object')

In [20]:
credits_df.id = credits_df.id.astype('int')
credits_df.dtypes

cast    object
crew    object
id       int64
dtype: object

In [21]:
keywords_df.id = keywords_df.id.astype('int')
keywords_df.dtypes

id           int64
keywords    object
dtype: object

In [22]:
movies_df.reset_index(inplace=True)

In [23]:
l=[]
for i in range(movies_df.shape[0]):
    if '-' in movies_df['id'][i]:
        print(i, movies_df['id'][i])
        l.append(i)

19730 1997-08-20
29503 2012-09-29
35587 2014-01-01


In [24]:
movies_df.drop(l, inplace=True)#.reset_index(inplace=True)

In [25]:
movies_df.id = movies_df.id.astype('int')
movies_df.dtypes

index                    int64
adult                   object
genres                  object
id                       int64
overview                object
production_companies    object
title                   object
dtype: object

In [26]:
movies_df.head(3)

,index,adult,genres,id,overview,production_companies,title
0,0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]",Toy Story
1,1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",Jumanji
2,2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,A family wedding reignites the ancient feud be...,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",Grumpier Old Men


In [27]:
credits_df.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [28]:
keywords_df.head(3)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


In [29]:
df = pd.merge(pd.merge(movies_df, credits_df, on="id"), keywords_df, on="id")

In [30]:
df.drop(columns=['index'], inplace=True)

In [31]:
df.shape

(46628, 9)

In [32]:
df.head()

,adult,genres,id,overview,production_companies,title,cast,crew,keywords
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]",Toy Story,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",Jumanji,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,A family wedding reignites the ancient feud be...,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",Grumpier Old Men,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,"Cheated on, mistreated and stepped on, the wom...",[{'name': 'Twentieth Century Fox Film Corporat...,Waiting to Exhale,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"[{'id': 35, 'name': 'Comedy'}]",11862,Just when George Banks has recovered from his ...,"[{'name': 'Sandollar Productions', 'id': 5842}...",Father of the Bride Part II,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


## Drop rows witth null values

In [33]:
df.isnull().sum()

adult                     0
genres                    0
id                        0
overview                995
production_companies      4
title                     4
cast                      0
crew                      0
keywords                  0
dtype: int64

In [34]:
df.isna().sum()

adult                     0
genres                    0
id                        0
overview                995
production_companies      4
title                     4
cast                      0
crew                      0
keywords                  0
dtype: int64

In [35]:
df.dropna(inplace=True)

In [36]:
df.isna().sum()

adult                   0
genres                  0
id                      0
overview                0
production_companies    0
title                   0
cast                    0
crew                    0
keywords                0
dtype: int64

### moddify generes
### from each dict we need to get the names and form a list

In [37]:
ast.literal_eval(df.genres[12])

[{'id': 10751, 'name': 'Family'},
 {'id': 16, 'name': 'Animation'},
 {'id': 12, 'name': 'Adventure'}]

In [38]:
def convert1(obj):
    L=[]
    for i in ast.literal_eval(obj):
        x = i['name'].lower().replace(" ", "")
        L.append(x)
    return L

In [39]:
df['genres'] = df['genres'].apply(convert1)

In [40]:
df.head(3)

,adult,genres,id,overview,production_companies,title,cast,crew,keywords
0,False,"[animation, comedy, family]",862,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]",Toy Story,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,"[adventure, fantasy, family]",8844,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",Jumanji,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"[romance, comedy]",15602,A family wedding reignites the ancient feud be...,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",Grumpier Old Men,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


### modify production_companies 

In [41]:
ast.literal_eval(df.production_companies[10])

[{'name': 'Columbia Pictures', 'id': 5},
 {'name': 'Castle Rock Entertainment', 'id': 97}]

In [42]:
df['production_companies'] = df['production_companies'].apply(convert1)

In [43]:
df.head(3)

,adult,genres,id,overview,production_companies,title,cast,crew,keywords
0,False,"[animation, comedy, family]",862,"Led by Woody, Andy's toys live happily in his ...",[pixaranimationstudios],Toy Story,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,"[adventure, fantasy, family]",8844,When siblings Judy and Peter discover an encha...,"[tristarpictures, teitlerfilm, interscopecommu...",Jumanji,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"[romance, comedy]",15602,A family wedding reignites the ancient feud be...,"[warnerbros., lancastergate]",Grumpier Old Men,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


### cast column modification 

In [44]:
ast.literal_eval(df.cast[10])

[{'cast_id': 1,
  'character': 'Andrew Shepherd',
  'credit_id': '52fe44dac3a36847f80adf79',
  'gender': 2,
  'id': 3392,
  'name': 'Michael Douglas',
  'order': 0,
  'profile_path': '/nmlitv4g2jXO3sJrnA9g980j2uc.jpg'},
 {'cast_id': 2,
  'character': 'Sydney Ellen Wade',
  'credit_id': '52fe44dac3a36847f80adf7d',
  'gender': 1,
  'id': 516,
  'name': 'Annette Bening',
  'order': 1,
  'profile_path': '/q8pCSQzGLrhgtVXBKzMnwfoeBlH.jpg'},
 {'cast_id': 3,
  'character': 'Lewis Rothschild',
  'credit_id': '52fe44dac3a36847f80adf81',
  'gender': 2,
  'id': 521,
  'name': 'Michael J. Fox',
  'order': 2,
  'profile_path': '/aeN1cA04vVXKrVy1SuS1JOhABBs.jpg'},
 {'cast_id': 4,
  'character': 'A.J. MacInerney',
  'credit_id': '52fe44dac3a36847f80adf85',
  'gender': 2,
  'id': 8349,
  'name': 'Martin Sheen',
  'order': 3,
  'profile_path': '/ve77Q75moQhoB6cxdMfigfrHeTA.jpg'},
 {'cast_id': 11,
  'character': 'Robin McCall',
  'credit_id': '52fe44dac3a36847f80adfad',
  'gender': 1,
  'id': 63279,
  '

In [45]:
def convert2(obj):
    L=[]
    for i in ast.literal_eval(obj):
        x = i['name'].lower().replace(" ", "")
        L.append(x)
    return L[:3]

In [46]:
df['cast'] = df['cast'].apply(convert2)

In [47]:
df.head(3)

,adult,genres,id,overview,production_companies,title,cast,crew,keywords
0,False,"[animation, comedy, family]",862,"Led by Woody, Andy's toys live happily in his ...",[pixaranimationstudios],Toy Story,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,"[adventure, fantasy, family]",8844,When siblings Judy and Peter discover an encha...,"[tristarpictures, teitlerfilm, interscopecommu...",Jumanji,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"[romance, comedy]",15602,A family wedding reignites the ancient feud be...,"[warnerbros., lancastergate]",Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


### crew column modification

In [48]:
ast.literal_eval(df.crew[10])

[{'credit_id': '52fe44dac3a36847f80adfa3',
  'department': 'Camera',
  'gender': 2,
  'id': 2702,
  'job': 'Director of Photography',
  'name': 'John Seale',
  'profile_path': '/w4wp04pRMqJI2vt3gFTdIWLDDzp.jpg'},
 {'credit_id': '52fe44dac3a36847f80adf8b',
  'department': 'Directing',
  'gender': 2,
  'id': 3026,
  'job': 'Director',
  'name': 'Rob Reiner',
  'profile_path': '/2zw9Iq9uo4vZiTQNQWdNFwbA1TA.jpg'},
 {'credit_id': '52fe44dac3a36847f80adf97',
  'department': 'Production',
  'gender': 2,
  'id': 3026,
  'job': 'Producer',
  'name': 'Rob Reiner',
  'profile_path': '/2zw9Iq9uo4vZiTQNQWdNFwbA1TA.jpg'},
 {'credit_id': '52fe44dac3a36847f80adfa9',
  'department': 'Editing',
  'gender': 2,
  'id': 3032,
  'job': 'Editor',
  'name': 'Robert Leighton',
  'profile_path': None},
 {'credit_id': '52fe44dac3a36847f80adf9d',
  'department': 'Sound',
  'gender': 2,
  'id': 9251,
  'job': 'Music',
  'name': 'Marc Shaiman',
  'profile_path': '/aesoPh8jPR5qDKjjt7o5uj9kS3W.jpg'},
 {'credit_id': '

In [49]:
def convert3(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'].lower().replace(" ", ""))
    
    return L

In [50]:
df['crew'] = df['crew'].apply(convert3)

In [51]:
df.head(3)

,adult,genres,id,overview,production_companies,title,cast,crew,keywords
0,False,"[animation, comedy, family]",862,"Led by Woody, Andy's toys live happily in his ...",[pixaranimationstudios],Toy Story,"[tomhanks, timallen, donrickles]",[johnlasseter],"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,"[adventure, fantasy, family]",8844,When siblings Judy and Peter discover an encha...,"[tristarpictures, teitlerfilm, interscopecommu...",Jumanji,"[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"[romance, comedy]",15602,A family wedding reignites the ancient feud be...,"[warnerbros., lancastergate]",Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


### modify keywords

In [52]:
ast.literal_eval(df['keywords'][3])

[{'id': 818, 'name': 'based on novel'},
 {'id': 10131, 'name': 'interracial relationship'},
 {'id': 14768, 'name': 'single mother'},
 {'id': 15160, 'name': 'divorce'},
 {'id': 33455, 'name': 'chick flick'}]

In [53]:
def convert4(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'].lower().replace(" ", ""))
    return L

In [54]:
df['keywords'] = df['keywords'].apply(convert4)

In [55]:
df.head(5)

,adult,genres,id,overview,production_companies,title,cast,crew,keywords
0,False,"[animation, comedy, family]",862,"Led by Woody, Andy's toys live happily in his ...",[pixaranimationstudios],Toy Story,"[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy, friendship, friends, riva..."
1,False,"[adventure, fantasy, family]",8844,When siblings Judy and Peter discover an encha...,"[tristarpictures, teitlerfilm, interscopecommu...",Jumanji,"[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbo..."
2,False,"[romance, comedy]",15602,A family wedding reignites the ancient feud be...,"[warnerbros., lancastergate]",Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger, ol..."
3,False,"[comedy, drama, romance]",31357,"Cheated on, mistreated and stepped on, the wom...",[twentiethcenturyfoxfilmcorporation],Waiting to Exhale,"[whitneyhouston, angelabassett, lorettadevine]",[forestwhitaker],"[basedonnovel, interracialrelationship, single..."
4,False,[comedy],11862,Just when George Banks has recovered from his ...,"[sandollarproductions, touchstonepictures]",Father of the Bride Part II,"[stevemartin, dianekeaton, martinshort]",[charlesshyer],"[baby, midlifecrisis, confidence, aging, daugh..."


### Modify 'adult' column

In [56]:
df.adult.unique()

array(['False', 'True'], dtype=object)

In [57]:
def convert5(obj):
    if obj == 'False':
        return ['f']
    elif obj == 'True':
        return ['t']
    else:
        return 'na'

In [58]:
df['adult'] = df.adult.apply(convert5)

In [59]:
df.head()

,adult,genres,id,overview,production_companies,title,cast,crew,keywords
0,[f],"[animation, comedy, family]",862,"Led by Woody, Andy's toys live happily in his ...",[pixaranimationstudios],Toy Story,"[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy, friendship, friends, riva..."
1,[f],"[adventure, fantasy, family]",8844,When siblings Judy and Peter discover an encha...,"[tristarpictures, teitlerfilm, interscopecommu...",Jumanji,"[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbo..."
2,[f],"[romance, comedy]",15602,A family wedding reignites the ancient feud be...,"[warnerbros., lancastergate]",Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger, ol..."
3,[f],"[comedy, drama, romance]",31357,"Cheated on, mistreated and stepped on, the wom...",[twentiethcenturyfoxfilmcorporation],Waiting to Exhale,"[whitneyhouston, angelabassett, lorettadevine]",[forestwhitaker],"[basedonnovel, interracialrelationship, single..."
4,[f],[comedy],11862,Just when George Banks has recovered from his ...,"[sandollarproductions, touchstonepictures]",Father of the Bride Part II,"[stevemartin, dianekeaton, martinshort]",[charlesshyer],"[baby, midlifecrisis, confidence, aging, daugh..."


### Modify 'overview' column

In [60]:
df['overview'] = df['overview'].apply(lambda x: x.split())

In [61]:
df.head(3)

,adult,genres,id,overview,production_companies,title,cast,crew,keywords
0,[f],"[animation, comedy, family]",862,"[Led, by, Woody,, Andy's, toys, live, happily,...",[pixaranimationstudios],Toy Story,"[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy, friendship, friends, riva..."
1,[f],"[adventure, fantasy, family]",8844,"[When, siblings, Judy, and, Peter, discover, a...","[tristarpictures, teitlerfilm, interscopecommu...",Jumanji,"[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbo..."
2,[f],"[romance, comedy]",15602,"[A, family, wedding, reignites, the, ancient, ...","[warnerbros., lancastergate]",Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger, ol..."


### Rearrange the columns

In [62]:
df.columns

Index(['adult', 'genres', 'id', 'overview', 'production_companies', 'title',
       'cast', 'crew', 'keywords'],
      dtype='object')

In [63]:
df = df[['id', 'title', 'adult', 'genres', 'overview', 'cast', 'crew', 'keywords', 'production_companies',]]
df.head(3)

,id,title,adult,genres,overview,cast,crew,keywords,production_companies
0,862,Toy Story,[f],"[animation, comedy, family]","[Led, by, Woody,, Andy's, toys, live, happily,...","[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy, friendship, friends, riva...",[pixaranimationstudios]
1,8844,Jumanji,[f],"[adventure, fantasy, family]","[When, siblings, Judy, and, Peter, discover, a...","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbo...","[tristarpictures, teitlerfilm, interscopecommu..."
2,15602,Grumpier Old Men,[f],"[romance, comedy]","[A, family, wedding, reignites, the, ancient, ...","[waltermatthau, jacklemmon, ann-margret]",[howarddeutch],"[fishing, bestfriend, duringcreditsstinger, ol...","[warnerbros., lancastergate]"


## Merge 3rd to last columns into a single column called 'tags'

In [64]:
df['tags'] = df['genres']+df['cast']+df['crew']+df['adult']+df['overview']+df['keywords']+df['production_companies']

In [65]:
df.head(2)

,id,title,adult,genres,overview,cast,crew,keywords,production_companies,tags
0,862,Toy Story,[f],"[animation, comedy, family]","[Led, by, Woody,, Andy's, toys, live, happily,...","[tomhanks, timallen, donrickles]",[johnlasseter],"[jealousy, toy, boy, friendship, friends, riva...",[pixaranimationstudios],"[animation, comedy, family, tomhanks, timallen..."
1,8844,Jumanji,[f],"[adventure, fantasy, family]","[When, siblings, Judy, and, Peter, discover, a...","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[boardgame, disappearance, basedonchildren'sbo...","[tristarpictures, teitlerfilm, interscopecommu...","[adventure, fantasy, family, robinwilliams, jo..."


In [66]:
df.columns

Index(['id', 'title', 'adult', 'genres', 'overview', 'cast', 'crew',
       'keywords', 'production_companies', 'tags'],
      dtype='object')

In [67]:
df = df.drop(columns=['adult', 'genres', 'overview', 'cast', 'crew','keywords', 'production_companies'])

In [68]:
df.head(3)

,id,title,tags
0,862,Toy Story,"[animation, comedy, family, tomhanks, timallen..."
1,8844,Jumanji,"[adventure, fantasy, family, robinwilliams, jo..."
2,15602,Grumpier Old Men,"[romance, comedy, waltermatthau, jacklemmon, a..."


In [69]:
df['tags'] = df['tags'].apply(lambda x:" ".join(x))

In [71]:
df = df.sample(10000)

In [73]:
df.reset_index(inplace=True)

In [76]:
df = df.drop(columns=['index'])

In [77]:
df.head()

,id,title,tags
0,212606,A Thousand Times Good Night,drama juliettebinoche nikolajcoster-waldau mar...
1,212748,Rhymes for Young Ghouls,drama roseannesupernault markantonykrupa dever...
2,10940,Babylon 5: In the Beginning,action drama sciencefiction adventure brucebox...
3,39993,Short Night of Glass Dolls,horror mystery thriller ingridthulin jeansorel...
4,14847,Miss Conception,comedy romance heathergraham miakirshner tomel...


In [78]:
df.dtypes

id        int64
title    object
tags     object
dtype: object

In [79]:
df.isna().sum()

id       0
title    0
tags     0
dtype: int64

## save the modified dataframe as csv file

In [80]:
df.to_csv('movies_tags.csv')

## stemming the tags

In [ ]:
!pip install nltk

In [86]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import *

porterStemmer = PorterStemmer()

def stemmer(sentence):
    wordList = nltk.TreebankWordTokenizer().tokenize(sentence)

    stemWords = [porterStemmer.stem(word) for word in wordList]

    return ' '.join(stemWords)

[nltk_data] Downloading package punkt to /home/tinku/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [87]:
sentence = df['tags'][0]
sentence

'drama juliettebinoche nikolajcoster-waldau mariadoylekennedy erikpoppe f On assignment while photographing a female suicide bomber in Kabul, Rebecca – one of the world’s top war photojournalists - gets badly hurt. Back home, another bomb drops as her husband and daughters give her an ultimatum: her work or her family. war photojournalist paradoxspillefilma/s zentropainternationalsweden newgrangepictures'

In [88]:
stemmer(sentence)

'drama juliettebinoch nikolajcoster-waldau mariadoylekennedi erikpopp f on assign while photograph a femal suicid bomber in kabul , rebecca – one of the world’ top war photojournalist - get badli hurt. back home , anoth bomb drop as her husband and daughter give her an ultimatum : her work or her family. war photojournalist paradoxspillefilma/ zentropainternationalsweden newgrangepictur'

In [89]:
df['tags'] = df['tags'].apply(stemmer)

## vectorize the tags column

- With this we can represent every movie as a vector on the n-D plane

In [ ]:
!pip install scikit-learn

In [90]:
from sklearn.feature_extraction.text import CountVectorizer

In [91]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [92]:
vector = cv.fit_transform(df['tags']).toarray()

In [93]:
vector.shape

(10000, 5000)

### Find the Angular distance

- With cosine_similarity we can find the Angular Distance between any two vectors
- Lesser the distance, greater the similarity

In [94]:
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
similarity = cosine_similarity(vector)

In [96]:
pd.DataFrame(similarity).head()

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,1.000000,0.059131,0.209061,0.000000,0.000000,0.000000,0.000000,0.047974,0.103418,0.114992,...,0.000000,0.020056,0.056980,0.048912,0.056980,0.021990,0.000000,0.000000,0.216612,0.046524
1,0.059131,1.000000,0.054393,0.000000,0.000000,0.067267,0.000000,0.062409,0.134535,0.074796,...,0.000000,0.026091,0.074125,0.000000,0.074125,0.028606,0.000000,0.000000,0.035223,0.060523
2,0.209061,0.054393,1.000000,0.000000,0.000000,0.000000,0.025976,0.066194,0.095130,0.132221,...,0.000000,0.055347,0.052414,0.089984,0.052414,0.020228,0.054393,0.043853,0.174347,0.042796
3,0.000000,0.000000,0.000000,1.000000,0.051503,0.134535,0.110208,0.156022,0.000000,0.149592,...,0.101274,0.026091,0.000000,0.190885,0.148250,0.143032,0.076923,0.124035,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.051503,1.000000,0.000000,0.049192,0.083569,0.000000,0.000000,...,0.000000,0.017469,0.148888,0.000000,0.049629,0.038306,0.000000,0.000000,0.000000,0.040522


### Using similarity we will find out any 5 movies with the greatest similarity value excluding the movie itself and recommend those 5 movies to the user

## Movie Recommendation function

In [97]:
movie_index = df[df['title'] == 'Eureka'].index[0]
sorted(list(enumerate(similarity[movie_index])), reverse=True, key= lambda x: x[1])[1:6]

[(7381, 0.3265986323710904),
 (4353, 0.2961744388795461),
 (8132, 0.2782433374561009),
 (785, 0.273861278752583),
 (3630, 0.2666666666666666)]

In [98]:
def recommend(movie):
    movie_index = df[df['title'] == movie].index[0]
    most_similar = sorted(list(enumerate(similarity[movie_index])), reverse=True, key= lambda x: x[1])
    for i in most_similar[1:6]:
        print(df.iloc[i[0]].title)

In [101]:
recommend('Deathlands')

Fullmetal Alchemist the Movie: Conqueror of Shamballa
Act Of Love
Asphalt Playground
The Tempest
The Tempest


## Pickling

In [100]:
import pickle
pickle.dump(df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))